In [1]:
%pip install mediapipe opencv-python

In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os


 ### 2. Draw Hands


In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

drawing_color = (255, 255, 255)
line_thickness = 6

cap = cv2.VideoCapture(0)

canvas = np.zeros((480, 640, 3), dtype=np.uint8)

# Define landmarks for each gesture
closed_fist_landmarks = [mp_hands.HandLandmark.THUMB_TIP,
                         mp_hands.HandLandmark.INDEX_FINGER_TIP,
                         mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
                         mp_hands.HandLandmark.RING_FINGER_TIP,
                         mp_hands.HandLandmark.PINKY_TIP]

open_palm_landmarks = [mp_hands.HandLandmark.THUMB_TIP,
                       mp_hands.HandLandmark.INDEX_FINGER_TIP,
                       mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
                       mp_hands.HandLandmark.RING_FINGER_TIP,
                       mp_hands.HandLandmark.PINKY_TIP]

pointing_up_landmarks = [mp_hands.HandLandmark.THUMB_TIP,
                         mp_hands.HandLandmark.INDEX_FINGER_TIP]

def is_closed_fist(hand_landmarks):
    for landmark in closed_fist_landmarks:
        if hand_landmarks.landmark[landmark].y < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y:
            return False
    return True

def is_open_palm(hand_landmarks):
    for landmark in open_palm_landmarks:
        if hand_landmarks.landmark[landmark].y > hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y:
            return False
    return True

def is_pointing_up(hand_landmarks):
    if (hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y and
        hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y):
        return True
    return False

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        frame = cv2.flip(frame, 1)

        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process hand landmarks
        results = hands.process(image)

        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Rendering results
        if results.multi_hand_landmarks:
            gesture_labels = []

            for hand_landmarks in results.multi_hand_landmarks:
                index_finger_landmark = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                x = int(index_finger_landmark.x * frame.shape[1])
                y = int(index_finger_landmark.y * frame.shape[0])

                cv2.circle(image, (x, y), 10, drawing_color, -1)
                cv2.circle(canvas, (x, y), line_thickness, drawing_color, -1)

                # Recognize closed fist gesture
                if is_closed_fist(hand_landmarks):
                    gesture_labels.append("Closed Fist")

                # Recognize open palm gesture
                if is_open_palm(hand_landmarks):
                    gesture_labels.append("Open Palm")

                # Recognize pointing up gesture
                if is_pointing_up(hand_landmarks):
                    gesture_labels.append("Pointing Up")

            # Display gesture labels
            label_y = 50
            for label in gesture_labels:
                cv2.putText(image, label, (30, label_y), cv2.FONT_HERSHEY_SIMPLEX, 1, drawing_color, 2)
                label_y += 50

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

        cv2.imshow('Hand Tracking', image)

cap.release()
cv2.destroyAllWindows()